In [4]:
import numpy as np # 数据处理最重要的模块
import pandas as pd # 数据处理最重要的模块

from IPython.core.interactiveshell import InteractiveShell # jupyter运行输出的模块

#显示每一个运行结果
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns', None)

In [5]:
%%time
data = pd.read_csv('"D:\class1\000001.csv"')
data['Day'] = pd.to_datetime(data['Day'],format='%Y/%m/%d')
data.set_index('Day', inplace = True)
data.sort_values(by = ['Day'],axis=0, ascending=True)
data

<unknown>:1: SyntaxWarning: invalid escape sequence '\c'


ValueError: embedded null character

In [6]:
data_new = data['1995-01':'2025-08'].copy()
data_new['Close'] = pd.to_numeric(data_new['Close'])
data_new['Preclose'] = pd.to_numeric(data_new['Preclose'])
data_new

NameError: name 'data' is not defined

In [ ]:
data_new['Raw_return'] = data_new['Close'] / data_new['Preclose'] - 1
data_new['Log_return'] = np.log(data_new['Close']) - np.log(data_new['Preclose'])

# 方法2：使用pandas的pct_change函数计算收益率（适用于时间序列数据）
# 注意：这种方法需要数据已经按时间排序
data_new['Pct_change_return'] = data_new['Close'].pct_change()

# 方法3：使用apply方法（不推荐，因为速度较慢）
data_new['Apply_return'] = data_new.apply(lambda row: row['Close'] / row['Preclose'] - 1, axis=1)

# 方法4：使用diff和div方法组合（另一种向量化操作）
data_new['Diff_div_return'] = data_new['Close'].diff() / data_new['Close'].shift(1)

# 比较不同方法计算结果的差异
data_new
# 获取所有列名
cols = data_new.columns.tolist()

# 找到 Raw_return 和 Log_return 的索引位置
raw_idx = cols.index('Raw_return')
log_idx = cols.index('Log_return')

# 交换这两个位置
cols[raw_idx], cols[log_idx] = cols[log_idx], cols[raw_idx]

# 重新排列 DataFrame 的列顺序
data_new = data_new[cols]

,Preclose,Open,Highest,Lowest,Close,Raw_return,Log_return,Pct_change_return,Apply_return,Diff_div_return
Day,,,,,,,,,,
1995-01-03,647.870,637.720,647.710,630.530,639.880,-0.012333,-0.012409,NaN,-0.012333,NaN
1995-01-04,639.880,641.900,655.510,638.860,653.810,0.021770,0.021536,0.021770,0.021770,0.021770
1995-01-05,653.810,655.380,657.520,645.810,646.890,-0.010584,-0.010641,-0.010584,-0.010584,-0.010584
1995-01-06,646.890,642.750,643.890,636.330,640.760,-0.009476,-0.009521,-0.009476,-0.009476,-0.009476
1995-01-09,640.760,637.520,637.550,625.040,626.000,-0.023035,-0.023305,-0.023035,-0.023035,-0.023035
...,...,...,...,...,...,...,...,...,...,...
2025-08-25,3825.759,3848.163,3883.562,3839.972,3883.562,0.015109,0.014996,0.015109,0.015109,0.015109
2025-08-26,3883.562,3871.471,3888.599,3859.758,3868.382,-0.003909,-0.003916,-0.003909,-0.003909,-0.003909
2025-08-27,3868.382,3869.612,3887.198,3800.350,3800.350,-0.017587,-0.017743,-0.017587,-0.017587,-0.017587


In [ ]:
# 创建新列存储结果
if 'Loop_return' not in data_new.columns:
    data_new['Loop_return'] = np.nan

# 使用for循环计算
for i in range(len(data_new)):
    data_new.iloc[i, data_new.columns.get_loc('Loop_return')] = data_new.iloc[i, data_new.columns.get_loc('Close')] / data_new.iloc[i, data_new.columns.get_loc('Preclose')] - 1

# 方法6：使用zip和enumerate组合（比纯for循环更Pythonic）
close_values = data_new['Close'].values
preclose_values = data_new['Preclose'].values
loop_return_values = []

for i, (close, preclose) in enumerate(zip(close_values, preclose_values)):
    if preclose != 0 and not np.isnan(preclose):
        loop_return_values.append(close / preclose - 1)
    else:
        loop_return_values.append(np.nan)

data_new['Loop_return2'] = loop_return_values

# 方法7：使用numpy的向量化操作（高效且简洁）
data_new['Numpy_return'] = (data_new['Close'].values / data_new['Preclose'].values) - 1

# 显示结果
data_new

C:\Users\sa'si'gei\AppData\Local\Temp\ipykernel_4100\168867736.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new['Loop_return'] = np.nan
C:\Users\sa'si'gei\AppData\Local\Temp\ipykernel_4100\168867736.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new['Loop_return2'] = loop_return_values
C:\Users\sa'si'gei\AppData\Local\Temp\ipykernel_4100\168867736.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Preclose,Open,Highest,Lowest,Close,Log_return,Raw_return,Pct_change_return,Apply_return,Diff_div_return,Loop_return,Loop_return2,Numpy_return
Day,,,,,,,,,,,,,
1995-01-03,647.870,637.720,647.710,630.530,639.880,-0.012409,-0.012333,NaN,-0.012333,NaN,-0.012333,-0.012333,-0.012333
1995-01-04,639.880,641.900,655.510,638.860,653.810,0.021536,0.021770,0.021770,0.021770,0.021770,0.021770,0.021770,0.021770
1995-01-05,653.810,655.380,657.520,645.810,646.890,-0.010641,-0.010584,-0.010584,-0.010584,-0.010584,-0.010584,-0.010584,-0.010584
1995-01-06,646.890,642.750,643.890,636.330,640.760,-0.009521,-0.009476,-0.009476,-0.009476,-0.009476,-0.009476,-0.009476,-0.009476
1995-01-09,640.760,637.520,637.550,625.040,626.000,-0.023305,-0.023035,-0.023035,-0.023035,-0.023035,-0.023035,-0.023035,-0.023035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-25,3825.759,3848.163,3883.562,3839.972,3883.562,0.014996,0.015109,0.015109,0.015109,0.015109,0.015109,0.015109,0.015109
2025-08-26,3883.562,3871.471,3888.599,3859.758,3868.382,-0.003916,-0.003909,-0.003909,-0.003909,-0.003909,-0.003909,-0.003909,-0.003909
2025-08-27,3868.382,3869.612,3887.198,3800.350,3800.350,-0.017743,-0.017587,-0.017587,-0.017587,-0.017587,-0.017587,-0.017587,-0.017587


In [ ]:
#月季度
Month_data1 = data_new.resample('ME')['Log_return'].sum().to_frame(name='Log_return') 
Month_data1['Raw_Return'] = np.exp(Month_data1['Log_return']) - 1

# 添加年月信息便于分析
Month_data1['Year'] = Month_data1.index.year
Month_data1['Month'] = Month_data1.index.month

# 显示结果
Month_data1.head()

,Log_return,Raw_Return,Year,Month
Day,,,,
1995-01-31,-0.141139,-0.131631,1995,1
1995-02-28,-0.023979,-0.023694,1995,2
1995-03-31,0.163651,0.177803,1995,3
1995-04-30,-0.109315,-0.103552,1995,4
1995-05-31,0.188901,0.207922,1995,5


In [ ]:
Month_data2 = data_new.resample('ME')['Close'].last().to_frame()
Month_data2['Preclose'] = Month_data2['Close'].shift(1)
Month_data2['Raw_return'] = Month_data2['Close'] / Month_data2['Preclose'] - 1
Month_data2['Log_return'] = np.log(Month_data2['Close']) - np.log(Month_data2['Preclose'])

# 添加年月信息
Month_data2['Year'] = Month_data2.index.year
Month_data2['Month'] = Month_data2.index.month

# 显示结果
Month_data2.head()

,Close,Preclose,Raw_return,Log_return,Year,Month
Day,,,,,,
1995-01-31,562.59,NaN,NaN,NaN,1995,1
1995-02-28,549.26,562.59,-0.023694,-0.023979,1995,2
1995-03-31,646.92,549.26,0.177803,0.163651,1995,3
1995-04-30,579.93,646.92,-0.103552,-0.109315,1995,4
1995-05-31,700.51,579.93,0.207922,0.188901,1995,5


In [ ]:
#groupy by
# “1990-12-12”日期格式 里面的year年份 month月份 day 直接提出取来
data_new2 = data_new.copy()
data_new2['year'] = data_new2.index.year
data_new2['month'] = data_new2.index.month
data_new2
# 使用的时间、日期格式提取 字符串提出的方式 前四个字符当作年份 6-7字符是月份 提取出来的是字符串 变成数值

,Preclose,Open,Highest,Lowest,Close,Log_return,Raw_return,Pct_change_return,Apply_return,Diff_div_return,Loop_return,Loop_return2,Numpy_return,year,month
Day,,,,,,,,,,,,,,,
1995-01-03,647.870,637.720,647.710,630.530,639.880,-0.012409,-0.012333,NaN,-0.012333,NaN,-0.012333,-0.012333,-0.012333,1995,1
1995-01-04,639.880,641.900,655.510,638.860,653.810,0.021536,0.021770,0.021770,0.021770,0.021770,0.021770,0.021770,0.021770,1995,1
1995-01-05,653.810,655.380,657.520,645.810,646.890,-0.010641,-0.010584,-0.010584,-0.010584,-0.010584,-0.010584,-0.010584,-0.010584,1995,1
1995-01-06,646.890,642.750,643.890,636.330,640.760,-0.009521,-0.009476,-0.009476,-0.009476,-0.009476,-0.009476,-0.009476,-0.009476,1995,1
1995-01-09,640.760,637.520,637.550,625.040,626.000,-0.023305,-0.023035,-0.023035,-0.023035,-0.023035,-0.023035,-0.023035,-0.023035,1995,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-25,3825.759,3848.163,3883.562,3839.972,3883.562,0.014996,0.015109,0.015109,0.015109,0.015109,0.015109,0.015109,0.015109,2025,8
2025-08-26,3883.562,3871.471,3888.599,3859.758,3868.382,-0.003916,-0.003909,-0.003909,-0.003909,-0.003909,-0.003909,-0.003909,-0.003909,2025,8
2025-08-27,3868.382,3869.612,3887.198,3800.350,3800.350,-0.017743,-0.017587,-0.017587,-0.017587,-0.017587,-0.017587,-0.017587,-0.017587,2025,8


In [ ]:
# 方法3：使用groupby函数按年月分组计算月度收益率
# 首先提取年月信息
data_new3 = data_new.copy()
data_new3['year'] = data_new3.index.year
data_new3['month'] = data_new3.index.month

# 使用groupby按年月分组，然后对每组的对数收益率求和
Month_data3 = data_new3.groupby(['year', 'month'])['Log_return'].sum().to_frame()
Month_data3['Raw_Return'] = np.exp(Month_data3['Log_return']) - 1

# 显示结果
Month_data3

Log_return  Raw_Return
year month                        
1995 1       -0.141139   -0.131631
     2       -0.023979   -0.023694
     3        0.163651    0.177803
     4       -0.109315   -0.103552
     5        0.188901    0.207922
...                ...         ...
2025 4       -0.017148   -0.017002
     5        0.020662    0.020877
     6        0.028547    0.028959
     7        0.036707    0.037389
     8        0.076666    0.079682

[368 rows x 2 columns]

In [ ]:
# 方法4：使用apply和lambda函数进行更灵活的分组计算
Month_data4 = pd.DataFrame(
    data_new3.groupby(['year', 'month'])['Log_return'].apply(lambda x: sum(x))
)
Month_data4.columns = ['Log_return_sum']
Month_data4['Raw_Return'] = np.exp(Month_data4['Log_return_sum']) - 1

# 方法5：使用agg函数同时计算多个统计量
Month_data5 = data_new3.groupby(['year', 'month']).agg({
    'Log_return': ['sum', 'mean', 'std', 'count'],
    'Raw_return': ['mean', 'std']
})

# 在Jupyter中直接显示DataFrame会呈现为表格
print("方法4结果:")
display(Month_data4.head())  # 使用display函数在Jupyter中呈现表格

print("\n方法5结果（包含多个统计量）:")
display(Month_data5.head())

方法4结果:


Log_return_sum  Raw_Return
year month                            
1995 1           -0.141139   -0.131631
     2           -0.023979   -0.023694
     3            0.163651    0.177803
     4           -0.109315   -0.103552
     5            0.188901    0.207922


方法5结果（包含多个统计量）:


Log_return                           Raw_return          
                  sum      mean       std count       mean       std
year month                                                          
1995 1      -0.141139 -0.007428  0.016251    19  -0.007277  0.016140
     2      -0.023979 -0.001411  0.033003    17  -0.000891  0.033609
     3       0.163651  0.007115  0.023204    23   0.007401  0.023470
     4      -0.109315 -0.005466  0.020374    20  -0.005255  0.020169
     5       0.188901  0.008586  0.077844    22   0.011645  0.083211

In [ ]:
# 计算季度收益率
# 方法1：使用resample函数的'QE'参数（季度末）
Quarter_data1 = data_new.resample('QE')['Log_return'].sum().to_frame(name='Log_return')
Quarter_data1['Raw_Return'] = np.exp(Quarter_data1['Log_return']) - 1
Quarter_data1['Year'] = Quarter_data1.index.year
Quarter_data1['Quarter'] = Quarter_data1.index.quarter

# 方法2：使用季度末价格计算
Quarter_data2 = data_new.resample('QE')['Close'].last().to_frame()
Quarter_data2['Preclose'] = Quarter_data2['Close'].shift(1)
Quarter_data2['Raw_return'] = Quarter_data2['Close'] / Quarter_data2['Preclose'] - 1
Quarter_data2['Log_return'] = np.log(Quarter_data2['Close']) - np.log(Quarter_data2['Preclose'])

# 显示结果
print("季度对数收益率汇总:")
Quarter_data1
print("\n季度末价格计算的收益率:")
Quarter_data2

季度对数收益率汇总:


,Log_return,Raw_Return,Year,Quarter
Day,,,,
1995-03-31,-0.001467,-0.001466,1995,1
1995-06-30,-0.025583,-0.025258,1995,2
1995-09-30,0.135980,0.145660,1995,3
1995-12-31,-0.263130,-0.231358,1995,4
1996-03-31,0.001979,0.001981,1996,1
...,...,...,...,...
2024-09-30,0.117234,0.124383,2024,3
2024-12-31,0.004565,0.004575,2024,4
2025-03-31,-0.004790,-0.004779,2025,1



季度末价格计算的收益率:


,Close,Preclose,Raw_return,Log_return
Day,,,,
1995-03-31,646.920,NaN,NaN,NaN
1995-06-30,630.580,646.9200,-0.025258,-0.025583
1995-09-30,722.430,630.5800,0.145660,0.135980
1995-12-31,555.290,722.4300,-0.231358,-0.263130
1996-03-31,556.390,555.2900,0.001981,0.001979
...,...,...,...,...
2024-09-30,3336.497,2967.4028,0.124383,0.117234
2024-12-31,3351.763,3336.4970,0.004575,0.004565
2025-03-31,3335.746,3351.7630,-0.004779,-0.004790


In [ ]:
#年季度
# 计算年度收益率
# 方法1：使用resample函数的'YE'参数（年末）
Year_data1 = data_new.resample('YE')['Log_return'].sum().to_frame(name='Log_return')
Year_data1['Raw_Return'] = np.exp(Year_data1['Log_return']) - 1

# 方法2：使用年末价格计算
Year_data2 = data_new.resample('YE')['Close'].last().to_frame()
Year_data2['Preclose'] = Year_data2['Close'].shift(1)
Year_data2['Raw_return'] = Year_data2['Close'] / Year_data2['Preclose'] - 1
Year_data2['Log_return'] = np.log(Year_data2['Close']) - np.log(Year_data2['Preclose'])

# 方法3：使用groupby按年分组
data_new4 = data_new.copy()
data_new4['year'] = data_new4.index.year
Year_data3 = data_new4.groupby('year')['Log_return'].sum().to_frame()
Year_data3['Raw_Return'] = np.exp(Year_data3['Log_return']) - 1

# 显示结果
print("年度对数收益率汇总:")
Year_data1
print("\n年末价格计算的收益率:")
Year_data2
print("\n使用groupby计算的年度收益率:")
Year_data3

年度对数收益率汇总:


,Log_return,Raw_Return
Day,,
1995-12-31,-0.154200,-0.142899
1996-12-31,0.501639,0.651425
1997-12-31,0.264019,0.302153
1998-12-31,-0.040505,-0.039695
1999-12-31,0.175423,0.191750
2000-12-31,0.416917,0.517277
2001-12-31,-0.230898,-0.206180
2002-12-31,-0.192575,-0.175167
2003-12-31,0.097735,0.102670



年末价格计算的收益率:


,Close,Preclose,Raw_return,Log_return
Day,,,,
1995-12-31,555.2900,NaN,NaN,NaN
1996-12-31,917.0200,555.2900,0.651425,0.501639
1997-12-31,1194.1000,917.0200,0.302153,0.264019
1998-12-31,1146.7000,1194.1000,-0.039695,-0.040505
1999-12-31,1366.5800,1146.7000,0.191750,0.175423
2000-12-31,2073.4800,1366.5800,0.517277,0.416917
2001-12-31,1645.9700,2073.4800,-0.206180,-0.230898
2002-12-31,1357.6500,1645.9700,-0.175167,-0.192575
2003-12-31,1497.0400,1357.6500,0.102670,0.097735



使用groupby计算的年度收益率:


,Log_return,Raw_Return
year,,
1995,-0.154200,-0.142899
1996,0.501639,0.651425
1997,0.264019,0.302153
1998,-0.040505,-0.039695
1999,0.175423,0.191750
2000,0.416917,0.517277
2001,-0.230898,-0.206180
2002,-0.192575,-0.175167
2003,0.097735,0.102670


In [ ]:
# 计算滚动收益率（例如：过去30天、60天、90天的收益率 注意这里指的是前30个观测值）
# 这在金融分析中非常常见，用于观察不同时间窗口的收益表现

# 方法1：使用rolling窗口函数计算滚动对数收益率之和
rolling_returns = pd.DataFrame()
for window in [5, 10, 20, 30, 60]:
    # 计算滚动窗口的对数收益率之和
    rolling_log_return = data_new['Log_return'].rolling(window=window).sum()
    # 转换为原始收益率
    rolling_returns[f'Rolling_{window}d_Return'] = np.exp(rolling_log_return) - 1

# 方法2：使用pct_change计算滚动价格变化
rolling_price_returns = pd.DataFrame()
for window in [5, 10, 20, 30, 60]:
    rolling_price_returns[f'Rolling_{window}d_Price_Return'] = data_new['Close'].pct_change(periods=window)

# 显示结果
print("滚动收益率 (基于对数收益率累加):")
print(rolling_returns.tail())
print("\n滚动收益率 (基于价格变化):")
print(rolling_price_returns.tail())

滚动收益率 (基于对数收益率累加):
            Rolling_5d_Return  Rolling_10d_Return  Rolling_20d_Return  \
Day                                                                     
2025-08-25           0.041720            0.064705            0.079386   
2025-08-26           0.037854            0.055229            0.071660   
2025-08-27           0.009065            0.031732            0.051064   
2025-08-28           0.019225            0.048318            0.075671   
2025-08-29           0.008408            0.043594            0.083702   

            Rolling_30d_Return  Rolling_60d_Return  
Day                                                 
2025-08-25            0.103394            0.160143  
2025-08-26            0.103676            0.150627  
2025-08-27            0.084644            0.125628  
2025-08-28            0.092916            0.135781  
2025-08-29            0.091511            0.139592  

滚动收益率 (基于价格变化):
            Rolling_5d_Price_Return  Rolling_10d_Price_Return  \
Day             

In [ ]:
# 计算累积收益率
# 累积收益率用于观察长期投资表现，从某个起始点开始累积

# 方法1：使用对数收益率累加后转换
# 这是最准确的方法，特别是对于长期累积
cumulative_returns = pd.DataFrame()
cumulative_returns['Cumulative_Log_Return'] = data_new['Log_return'].cumsum()
cumulative_returns['Cumulative_Return'] = np.exp(cumulative_returns['Cumulative_Log_Return']) - 1

# 方法2：使用cumprod函数直接累乘(1+r)
# 这种方法在金融实践中也很常见
cumulative_returns['Cumulative_Return_Prod'] = (1 + data_new['Raw_return']).cumprod() - 1

# 方法3：使用pandas的累积函数
cumulative_returns['Cumulative_Return_Alt'] = data_new['Raw_return'].add(1).cumprod().sub(1)

# 显示结果
print("不同方法计算的累积收益率:")
cumulative_returns

不同方法计算的累积收益率:


,Cumulative_Log_Return,Cumulative_Return,Cumulative_Return_Prod,Cumulative_Return_Alt
Day,,,,
1995-01-03,-0.012409,-0.012333,-0.012333,-0.012333
1995-01-04,0.009127,0.009169,0.009169,0.009169
1995-01-05,-0.001514,-0.001513,-0.001513,-0.001513
1995-01-06,-0.011035,-0.010974,-0.010974,-0.010974
1995-01-09,-0.034340,-0.033757,-0.033757,-0.033757
...,...,...,...,...
2025-08-25,1.790819,4.994361,4.994361,4.994361
2025-08-26,1.786903,4.970931,4.970931,4.970931
2025-08-27,1.769160,4.865922,4.865922,4.865922
